In [81]:
import torch
import torch.nn as nn
import json
import numpy as np

In [82]:
# # 读取json文件内容,返回字典格式
# with open('unlabeled.json','r')as fp:
#     unlabeled_ann_data = json.load(fp)
#     print('这是文件中的json数据：',unlabeled_ann_data[0]['id'])
#     print('这是读取到文件数据的数据类型：', type(unlabeled_ann_data))

# with open('labeled.json','r')as fp:
#     labeled_ann_data = json.load(fp)
#     print('这是文件中的json数据：',labeled_ann_data[0]['id'])
#     print('这是读取到文件数据的数据类型：', type(labeled_ann_data))

In [83]:
# print(len(unlabeled_ann_data))
# print(len(labeled_ann_data))
# unlabeled_ann_data[0]['id']
# print(unlabeled_ann_data[0:5])

In [84]:
input_dim = 1
batch_size = 64
output_dim = 200
unlabel_data_num = 1000000
for_num = int(unlabel_data_num/batch_size)

model = nn.Sequential(nn.Linear(input_dim, output_dim),
                      nn.Softmax(dim=1))

In [85]:
# with open('labeled.json','r')as fp:
#     json_data = json.load(fp)
#     print('这是文件中的json数据：',json_data[0]['id'])
#     print('这是读取到文件数据的数据类型：', type(json_data))


In [86]:
input_test = torch.rand(batch_size,input_dim)
output = model(input_test)

In [87]:
print(output.shape)
print(output)

torch.Size([64, 200])
tensor([[0.0062, 0.0018, 0.0062,  ..., 0.0023, 0.0106, 0.0024],
        [0.0052, 0.0016, 0.0075,  ..., 0.0014, 0.0131, 0.0029],
        [0.0067, 0.0020, 0.0054,  ..., 0.0030, 0.0092, 0.0021],
        ...,
        [0.0066, 0.0019, 0.0056,  ..., 0.0028, 0.0095, 0.0022],
        [0.0063, 0.0019, 0.0060,  ..., 0.0024, 0.0103, 0.0023],
        [0.0068, 0.0020, 0.0053,  ..., 0.0032, 0.0089, 0.0020]],
       grad_fn=<SoftmaxBackward>)


In [95]:
input_test = torch.rand(batch_size,input_dim)
#strategy1:选择输出概率最大的类别作为伪标签
semi_label = output.argmax(dim=1)
# print(semi_label)

#strategy2:按多项式分布抽取伪标签
semi_label = torch.multinomial(output, 1, replacement=False).squeeze()
# print(semi_label)


#strategy3:按先验阈值选取伪标签
threshold = 0.0165
is_labeled = torch.zeros(1000000)
semi_label = []
for i in range(for_num):
    output = model(input_test)
    max_pro = output.max(dim=1)
    semi_label_index = max_pro.values>threshold
    # print(semi_label_index)
    # print(output[semi_label_index].shape)
    output_using = output[semi_label_index]
    is_labeled[batch_size*i:batch_size*(i+1)] = semi_label_index
    semi_label+=output_using.argmax(dim=1).tolist()

In [96]:

print(len(semi_label))

265625
